In [0]:
# 匯入「keras_lite_convertor.py」到 Colab

from google.colab import files
uploaded = files.upload()       # 匯入檔案

In [0]:
# 讀取 voice_data 資料夾內的檔案

import keras_lite_convertor as kc

path_name = 'voice_data'   
Data_reader = kc.Data_reader(path_name, 
                             mode='categorical',     # categorical 適用於多元分類
                             label_name = ['on','off','others'])   # 標籤名稱
data, label = Data_reader.read()

In [0]:
# 資料預處理

# 取資料中的 90% 當作訓練集
split_num = int(len(data)*0.9) 
train_data=data[:split_num]
train_label=label[:split_num]    

# 正規化
mean = train_data.mean() # 平均數
data -= mean
std = train_data.std()   # 標準差
data /= std

In [0]:
# 訓練集、驗證集、測試集的資料形狀

# 訓練集
print(train_data.shape)    

# 驗證集
validation_data=data[split_num:-5]  
print(validation_data.shape)
validation_label=label[split_num:-5]   

# 測試集
test_data=data[-5:]     
print(test_data.shape)
test_label=label[-5:]

(270, 400)
(25, 400)
(5, 400)


In [0]:
# 建立卷積神經網路

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

model = Sequential()
model.add(layers.Reshape((400,1), input_shape=(400,)))
model.add(layers.Conv1D(4,3,activation = 'relu',      # 卷積層
                        padding='valid'))
model.add(layers.MaxPooling1D())                      # 池化層
model.add(layers.Conv1D(4,3,activation = 'relu',
                        padding='valid'))
model.add(layers.MaxPooling1D())
model.add(layers.Conv1D(8,3,activation = 'relu',
                        padding='valid'))
model.add(layers.MaxPooling1D())
model.add(layers.Flatten())   # 展平層
model.add(layers.Dense(3,activation = 'softmax'))# 輸出層的啟動函數為 softmax

# 顯示模型資訊
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 400, 1)            0         
_________________________________________________________________
conv1d (Conv1D)              (None, 398, 4)            16        
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 199, 4)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 197, 4)            52        
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 98, 4)             0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 96, 8)             104       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 48, 8)             0

In [0]:
# 編譯及訓練模型

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
train_history = model.fit(train_data,train_label,                              # 訓練集
                          validation_data=(validation_data,validation_label),  # 驗證集
                          epochs=60)   # 訓練週期

Epoch 1/60
9/9 [==============================] - 0s 51ms/step - loss: 1.1328 - acc: 0.3778 - val_loss: 1.0945 - val_acc: 0.3200
Epoch 2/60
9/9 [==============================] - 0s 12ms/step - loss: 1.0571 - acc: 0.4370 - val_loss: 1.0319 - val_acc: 0.5600
Epoch 3/60
9/9 [==============================] - 0s 11ms/step - loss: 0.9864 - acc: 0.6185 - val_loss: 0.9666 - val_acc: 0.7600
Epoch 4/60
9/9 [==============================] - 0s 11ms/step - loss: 0.9068 - acc: 0.7111 - val_loss: 0.8861 - val_acc: 0.7200
Epoch 5/60
9/9 [==============================] - 0s 11ms/step - loss: 0.8171 - acc: 0.7704 - val_loss: 0.7995 - val_acc: 0.8400
Epoch 6/60
9/9 [==============================] - 0s 11ms/step - loss: 0.7170 - acc: 0.8074 - val_loss: 0.6925 - val_acc: 0.8800
Epoch 7/60
9/9 [==============================] - 0s 11ms/step - loss: 0.6176 - acc: 0.8593 - val_loss: 0.5902 - val_acc: 0.9200
Epoch 8/60
9/9 [==============================] - 0s 11ms/step - loss: 0.5197 - acc: 0.9037 - val

In [0]:
# 測試模型

print('predict:')
print(model.predict(test_data))
print()
print('real:')
print(test_label)

predict:
[[2.0708303e-05 9.9994206e-01 3.7170194e-05]
 [6.1850733e-05 9.9990249e-01 3.5608984e-05]
 [9.6273176e-02 8.6310470e-01 4.0622141e-02]
 [9.9651766e-01 2.6654743e-04 3.2157598e-03]
 [6.4241858e-03 8.9158374e-04 9.9268430e-01]]

real:
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]


In [0]:
# 儲存模型

kc.save(model, 'voice_model.json')

In [0]:
# 顯示資訊

print('mean=', mean)
print('std=', std)

mean= 112.06158333333333
std= 245.64491731346183
